In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
data=pd.read_csv(r'C:\Users\ELCOT\Downloads\santander-customer-satisfaction\train.csv')

In [3]:
data.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [4]:
data.isnull().sum()

ID                         0
var3                       0
var15                      0
imp_ent_var16_ult1         0
imp_op_var39_comer_ult1    0
                          ..
saldo_medio_var44_hace3    0
saldo_medio_var44_ult1     0
saldo_medio_var44_ult3     0
var38                      0
TARGET                     0
Length: 371, dtype: int64

In [5]:
data.shape

(76020, 371)

In [11]:
[col for col in data.columns if data[col].isnull().sum() > 0]

[]

In [12]:
X=data.drop(['TARGET'],axis=1)
y=data['TARGET']

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [14]:
X_train_original=X_train.copy()
X_test_original=X_test.copy()

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
def run_randomForests(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train Set')
    pred = rf.predict_proba(X_train)
    print("Train Results roc-auc {}".format(roc_auc_score(y_train, pred[:,1])))
    print('Test Set')
    pred= rf.predict_proba(X_test)
    print("Test Results roc-auc {}".format(roc_auc_score(y_test, pred[:,1])))

# 1. Constant features - numerical


In [18]:
from sklearn.feature_selection import VarianceThreshold
sel_ = VarianceThreshold(threshold=0)
sel_.fit(X_train)

VarianceThreshold(threshold=0)

In [19]:
#get_support() is a boolean vector that indicates which features are retained
sum(sel_.get_support())

335

In [21]:
print([x for x in X_train.columns if x not in X_train.columns[sel_.get_support()]])

['ind_var2_0', 'ind_var2', 'ind_var27_0', 'ind_var28_0', 'ind_var28', 'ind_var27', 'ind_var41', 'ind_var46_0', 'ind_var46', 'num_var27_0', 'num_var28_0', 'num_var28', 'num_var27', 'num_var41', 'num_var46_0', 'num_var46', 'saldo_var28', 'saldo_var27', 'saldo_var41', 'saldo_var46', 'imp_amort_var18_hace3', 'imp_amort_var34_hace3', 'imp_reemb_var13_hace3', 'imp_reemb_var33_hace3', 'imp_trasp_var17_out_hace3', 'imp_trasp_var33_out_hace3', 'num_var2_0_ult1', 'num_var2_ult1', 'num_reemb_var13_hace3', 'num_reemb_var33_hace3', 'num_trasp_var17_out_hace3', 'num_trasp_var33_out_hace3', 'saldo_var2_ult1', 'saldo_medio_var13_medio_hace3', 'saldo_medio_var29_hace3']


In [22]:
features_to_keep = X_train.columns[sel_.get_support()]

In [23]:
X_train = sel_.transform(X_train)
X_test = sel_.transform(X_test)

print(X_train.shape, X_test.shape)

(57015, 335) (19005, 335)


In [28]:

#Original data
run_randomForests(X_train_original, X_test_original, y_train, y_test)

Train Set
Train Results roc-auc 0.8055608468544493
Test Set
Test Results roc-auc 0.7782419474740464


In [30]:
# constant values removal
run_randomForests(X_train, X_test, y_train, y_test)

Train Set
Train Results roc-auc 0.8040622797405381
Test Set
Test Results roc-auc 0.7758331567294428


# 2.Quasi-constant features

In [34]:
X_train = pd.DataFrame(X_train)
X_train.columns = features_to_keep
X_test = pd.DataFrame(X_test)
X_test.columns = features_to_keep

In [35]:
sel1_ = VarianceThreshold(threshold=0.01)
sel1_.fit(X_train)

VarianceThreshold(threshold=0.01)

In [36]:
sum(sel1_.get_support())

271

In [37]:
print([x for x in X_train.columns if x not in X_train.columns[sel1_.get_support()]])

['ind_var1', 'ind_var6_0', 'ind_var6', 'ind_var13_largo_0', 'ind_var13_largo', 'ind_var13_medio_0', 'ind_var13_medio', 'ind_var14', 'ind_var17_0', 'ind_var17', 'ind_var18_0', 'ind_var18', 'ind_var19', 'ind_var20_0', 'ind_var20', 'ind_var29_0', 'ind_var29', 'ind_var30_0', 'ind_var31_0', 'ind_var31', 'ind_var32_cte', 'ind_var32_0', 'ind_var32', 'ind_var33_0', 'ind_var33', 'ind_var34_0', 'ind_var34', 'ind_var40', 'ind_var39', 'ind_var44_0', 'ind_var44', 'num_var6_0', 'num_var6', 'num_var13_medio_0', 'num_var13_medio', 'num_var18_0', 'num_var18', 'num_op_var40_hace3', 'num_var29_0', 'num_var29', 'num_var33', 'num_var34_0', 'num_var34', 'ind_var7_emit_ult1', 'ind_var7_recib_ult1', 'num_aport_var33_hace3', 'num_aport_var33_ult1', 'num_var7_emit_ult1', 'num_compra_var44_hace3', 'num_meses_var13_medio_ult3', 'num_meses_var17_ult3', 'num_meses_var29_ult3', 'num_meses_var33_ult3', 'num_meses_var44_ult3', 'num_reemb_var13_ult1', 'num_reemb_var17_hace3', 'num_reemb_var33_ult1', 'num_trasp_var17_in

In [38]:
features_to_keep = X_train.columns[sel1_.get_support()]

In [39]:
X_train = sel1_.transform(X_train)
X_test = sel1_.transform(X_test)
print(X_train.shape, X_test.shape)

(57015, 271) (19005, 271)


In [40]:
#Original data
run_randomForests(X_train_original, X_test_original, y_train, y_test)

Train Set
Train Results roc-auc 0.8055608468544493
Test Set
Test Results roc-auc 0.7782419474740464


In [41]:
run_randomForests(X_train, X_test, y_train, y_test)

Train Set
Train Results roc-auc 0.8087064513342502
Test Set
Test Results roc-auc 0.7807608009284976


# 3.duplicate features removal

In [42]:
X_train = pd.DataFrame(X_train)
X_train.columns = features_to_keep
X_test = pd.DataFrame(X_test)
X_test.columns = features_to_keep


In [43]:
duplicated_feat = []
for i in range(0, len(X_train.columns)):
    col_1 = X_train.columns[i]
    for col_2 in X_train.columns[i+1:]:
        if X_train[col_1].equals(X_train[col_2]):
            duplicated_feat.append(col_2)
            
print(duplicated_feat)

['ind_var26', 'ind_var25', 'ind_var37', 'num_var26', 'num_var25', 'num_var32', 'num_var37', 'num_var39', 'saldo_var29', 'saldo_medio_var13_medio_ult1', 'delta_num_reemb_var13_1y3', 'delta_num_reemb_var17_1y3', 'delta_num_reemb_var33_1y3', 'delta_num_trasp_var17_in_1y3', 'delta_num_trasp_var17_out_1y3', 'delta_num_trasp_var33_in_1y3', 'delta_num_trasp_var33_out_1y3']


In [44]:
print(len(set(duplicated_feat)))

17


In [45]:
X_train.drop(labels=duplicated_feat, axis=1, inplace=True)
X_test.drop(labels=duplicated_feat, axis=1, inplace=True)

print(X_train.shape, X_test.shape)

(57015, 254) (19005, 254)


In [46]:
#Original data
run_randomForests(X_train_original, X_test_original, y_train, y_test)

Train Set
Train Results roc-auc 0.8055608468544493
Test Set
Test Results roc-auc 0.7782419474740464


In [47]:
#Duplicate values Removal
run_randomForests(X_train, X_test, y_train, y_test)

Train Set
Train Results roc-auc 0.8056385105307589
Test Set
Test Results roc-auc 0.7785703731615031
